# House Prices

This project involved building an interactive web app, using Shiny for Python, to display house prices in England and Wales, adding a Choropleth layer to visualise the differences by region. The aim was to demonstrate the Shiny software and investigate trends in house prices over time and across different regions of England and Wales. This notebook contains is analysis into the data and a walkthrough of how the app was built, but if you'd just like to view the app then use this link:

**link**

##### About the data

For this project I used the Price Paid Data published monthly by the UK government, which tracks property sales in England and Wales submitted to HM Land Registry for registration. The dataset contains single residential properties sold for value since 1995, and since 2013 includes transfers under power of sale/repossessions, buy-to-lets (where identifiable by Mortgage type) and transfers to non-private individuals. This data is also used by property websites such as Zoopla and Rightmove to allow users to track property prices.

The Price Paid Data includes information on all residential property sales in England and Wales that are sold for value and are lodged with us for registration. It excludes:

- sales that have not been lodged with HM Land Registry
- sales that were not for value
- transfers, conveyances, assignments or leases at a premium with nominal rent, which are:
- ‘Right to buy’ sales at a discount
- subject to an existing mortgage
- to effect the sale of a share in a property, for example, a transfer between parties on divorce
- by way of a gift
- under a compulsory purchase order
- under a court order
- to Trustees appointed under Deed of appointment
- Vesting Deeds Transmissions or Assents of more than one property

##### Initial planning

- What granularity to use? e.g. county, town, postcode
    - The dataset contains the postcode of each property, and every postcode has a clearly defined area, which should be available online
    - The whole postcode is likely too granular - using the area code or district code should be a good compromise
- How to aggregate price paid by area?
    - Min, Max, Median, Mean
    - Allow user to choose which statistic to show on the map
    - Could also compare these summary statistics between points in time, which would highlight the areas which have seen the greatest changes in price over time

##### Preparing the data

Loading the CSV file into a MySQL database:

~~~~sql
DROP DATABASE IF EXISTS `houseprices`;
CREATE DATABASE `houseprices`;
USE `houseprices`;

CREATE TABLE `pricePaid` (
`unique_id` VARCHAR(100),
`price_paid` DECIMAL,
`deed_date` DATE,
`postcode` VARCHAR(8),
`property_type` VARCHAR(1),
`new_build` VARCHAR(1),
`estate_type` VARCHAR(1),
`saon` VARCHAR(50),
`paon` VARCHAR(50),
`street` VARCHAR(50),
`locality` VARCHAR(50),
`town` VARCHAR(50),
`district` VARCHAR(50),
`county` VARCHAR(50),
`transaction_category` VARCHAR(1),
`linked_data_uri` VARCHAR(1),
PRIMARY KEY (unique_id)
);

SET GLOBAL local_infile=ON;
SET autocommit=0;
SET unique_checks=1;
SET foreign_key_checks=0;

LOAD DATA LOW_PRIORITY 
LOCAL INFILE 'Path/To/Project/pricepaid.csv'
INTO TABLE pricePaid 
CHARACTER SET armscii8
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n' 
(`unique_id`,`price_paid`,`deed_date`,`postcode`,`property_type`,`new_build`,`estate_type`,`saon`,`paon`,`street`,`locality`,`town`,`district`,`county`,`transaction_category`,`linked_data_uri`);
~~~~

I had originally planned to use the full dataset in my Shiny app, however the full table is ~5GB in size with ~29m rows. For EDA purposes I am taking a sample of the data so that I can easily load the data into memory in Python. Taking a simple random sample of the data would mean that the number of samples from each area would be proportional to the population of that area, so to ensure that each area had an equal number of samples I would use stratified sampling instead.

I needed to choose a level of granularity to which to stratify the data. A UK postcode is made up of 2 parts, the outward code (first part) and inward code (second part), separated by a space. The outward code consists of the postcode area (either 1 or 2 letters) followed by the postcode district (usually 1 or 2 digits). For example, in the postcode PO16 7GZ, PO16 is the outward code (or outcode), PO is the area and 16 is the district.

OutCode was added as a generated column to the pricePaid table, along with a Year column and a YearBin column.

~~~~sql
ALTER TABLE pricePaid ADD COLUMN OutCode VARCHAR(4) GENERATED ALWAYS AS substr(postcode, 1, locate(' ', postcode) - 1) STORED;
ALTER TABLE pricePaid ADD COLUMN Year INT GENERATED ALWAYS AS year(cast(deed_date as date)) STORED;
ALTER TABLE pricePaid ADD COLUMN YearBin VARCHAR(4) GENERATED ALWAYS AS case when (Year < 2005) then '1995 - 2004' when (Year < 2015) then '2005 - 2014' else '2015 +' end STORED;
~~~~

Creating an index on Outcode and YearBin to speed up the stratified sample query.

~~~~sql
CREATE INDEX OutcodeYearBinIndex ON pricepaid (Outcode, YearBin);
~~~~

Taking a stratified sample of 100 observations for each distinct OutCode and YearBin. This is to be used for EDA.

~~~~sql
SELECT t.* FROM
(SELECT pp.*, ROW_NUMBER() OVER (PARTITION BY OutCode, YearBin ORDER BY RAND()) AS SeqNum
FROM pricePaid pp) t
WHERE t.SeqNum <= 100
INTO LOCAL OUTFILE '/Path/To/Project/pricepaidsample.csv'
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n';
~~~~

##### EDA

Importing the sample into Python and creating a density plot of all house prices:

In [ ]:
from pathlib import Path
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

### Load data
appDir = Path(os.path.abspath(''))
dataset = pd.read_csv(appDir / "pricepaidsample.csv", delimiter='\t', header=0, encoding="utf-8")
# remove missing postcodes and price paid values of zero
dataset = dataset[~((dataset['postcode'].isnull()) | (dataset['price_paid'] == 0))]
dataset['price_paid_mil'] = dataset['price_paid'] / 1000000

# density plot of price paid
fig, ax = plt.subplots()
sns.kdeplot(df['price_paid_mil'])
ax.set_xlabel('Price Paid (£m)')
ax.set_xlim(left=-5)
ax.set_ylim(bottom=-0.00005)
ax.set_title('Density Plot of Price Paid Sample (£m)')
ax.tick_params(bottom=True, left=True)
plt.show()

![EDADensityPlot](../images/EDADensityPlot.PNG)

This distribution is heavily right skewed due to some houses selling for 100s of millions. Looking at the summary statistics for the dataset indicates that the vast majority of price_paid values are less than £1m, so filtering out values greater than this should give a better view of the distribution.

In [ ]:
dataset.describe()

![summarystats](../images/summarystats.PNG)

Applying this <= £1m filter and also splitting the data by region to produce a ridge plot so that we compare the distributions by region:

In [ ]:
### postcode area to region lookup
outcodeRegion = pd.read_csv(appDir / "postcode_region_lookup.csv")
outcodeRegion['Area'] = outcodeRegion['Postcode prefix']
dataset['Area'] = dataset['Outcode'].replace('\d+', '', regex=True)
df = pd.merge(dataset, outcodeRegion, on="Area", how="left")

# filtering data for house prices <= £1m
df_filtered = df[df['price_paid_mil'] <= 1]

# ridge plot of log price paid by region
def ridgePlot(df, xVar, groups):
    sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
    palette = sns.color_palette("Set2", 12)
    g = sns.FacetGrid(df, palette=palette, row=groups, hue=groups, aspect=9, height=1.2)
    g.map_dataframe(sns.kdeplot, x=xVar, fill=True, alpha=1)
    g.map_dataframe(sns.kdeplot, x=xVar, color='black')

    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .2, label, color='black', fontsize=13,
                ha="left", va="center", transform=ax.transAxes)

    g.map(label, groups)
    g.figure.subplots_adjust(hspace=-.5)
    g.set_titles("")
    g.set(yticks=[], xlabel="Price Paid £m", ylabel="")
    g.despine(left=True)

ridgePlot(df_filtered, "price_paid_mil", "UK region")

![EDARidgePlot](../images/EDARidgePlot.PNG)

This shows that all regions have the same type of distribution - though some regions, such as London and South East, are more heavily right skewed. Next I am trying to see if I can fit this distribution to a known distribution, specifically the lognormal distribution (a distribution is lognormal if its log follow a normal distribution):

In [ ]:
import scipy
import numpy as np

### determining the distribution type
shape, location, scale = scipy.stats.lognorm.fit(df_filtered['price_paid_mil'])
mu, sigma = np.log(scale), shape
xmin, xmax = np.min(df_filtered['price_paid_mil']), np.max(df_filtered['price_paid_mil'])
x = np.linspace(xmin, xmax, 1000)

fig, ax = plt.subplots()
plt.plot(x, scipy.stats.lognorm.pdf(x, shape, location, scale), label="Lognormal")
sns.kdeplot(df_filtered, x="price_paid_mil", label="Density Plot")
ax.set_xlabel('Price Paid (£m)')
ax.set_ylim(bottom=-0.1)
ax.set_title('Price Paid Sample (£m) - Density Plot vs Fitted Lognormal Distribution')
ax.tick_params(bottom=True, left=True)
plt.legend(facecolor='white')
plt.show()

![EDAFitToLognormal](../images/EDAFitToLognormal.PNG)

Except for some of the data points at the peak of the curve, this appears to give a very close fit. We could conclude from this that the overall distribution of house prices less than £1m closely fits a lognormal distribution. We can check this with a QQ plot of the log of house price against a reference normal distribution:

In [ ]:
import statsmodels.api as sm

df_filtered['log_price_paid_mil'] = np.log(df_filtered['price_paid_mil'])

### QQ plot
fig, ax = plt.subplots(2, 1)
sm.qqplot(data=df_filtered['log_price_paid_mil'], line='s', ax=ax[0])
sns.kdeplot(df_filtered['log_price_paid_mil'], ax=ax[1])
plt.show()

![EDAQQPlot](../images/EDAQQPlot.PNG)

The log of price paid closely matches the normal distribution in the centre, but deviates significantly in the tails, suggesting that the data does not actually quite follow a lognormal distribution. Therefore, instead of using the lognormal parameters as summary statistics in my visualisation to describe the data, non-parametric statistics such as the median and IQR will be more appropriate.

##### Transforming and aggregating the data

Before the data can be used in the Shiny app, it needs to be aggregated. I'm choosing to aggregate by Outcode since these areas on the map should granular enough to give a detailed spatial distribution on the map, and each outcode should be of a similar size. Doing this outside the Shiny app and reading the aggregated data directly instead greatly improves the performance of the app. I am aggregating by Outcode and YearBin and calculating summary statistics - the summary statistics are chosen taking into consideration the highly right skewed shape of the pseudo-lognormal distribution - the median will be a better estimate of the average value than the arithmetic mean (because it's not affected by outliers), and the IQR will be a better estimate of spread for the same reason. I am also including skew since we know that the distributions of some regions are more heavily skewed than others.

~~~~sql
CREATE TABLE minMaxMeanCountStddev AS
SELECT Outcode, YearBin
,MIN(price_paid) AS min
,MAX(price_paid) AS max
,AVG(price_paid) AS mean
,STD(price_paid) AS stddev
,COUNT(*) AS count
FROM pricePaid
GROUP BY Outcode, YearBin;

CREATE TABLE skew AS
SELECT t1.Outcode, t1.YearBin
,SUM(POWER((price_paid - t1.mean), 3) / ((t1.count - 1) * POWER(t1.stddev, 3))) as skew
FROM minMaxMeanCountStddev t1
INNER JOIN pricePaid pp on t1.Outcode = pp.Outcode and t1.YearBin = pp.YearBin
GROUP BY t1.Outcode, t1.YearBin;

CREATE TABLE median AS
SELECT Outcode, YearBin
,AVG(price_paid) AS median
FROM (
SELECT Outcode, YearBin
,price_paid
,ROW_NUMBER() OVER (PARTITION BY Outcode, YearBin ORDER BY price_paid) rn
,COUNT(*) OVER (PARTITION BY Outcode, YearBin) cnt
FROM pricePaid
) dd
WHERE rn IN (FLOOR((cnt + 1) / 2), FLOOR((cnt + 2) / 2))
GROUP BY Outcode, YearBin;

CREATE TABLE lowerQuartile AS
SELECT Outcode, YearBin
,AVG(price_paid) AS lowerQuartile
FROM (
SELECT Outcode, YearBin
,price_paid
,ROW_NUMBER() OVER (PARTITION BY Outcode, YearBin ORDER BY price_paid) rn
,COUNT(*) OVER (PARTITION BY Outcode, YearBin) cnt
FROM pricePaid
) dd
WHERE rn IN (FLOOR((cnt + 1) / 4), FLOOR((cnt + 2) / 4))
GROUP BY Outcode, YearBin;

CREATE TABLE upperQuartile AS
SELECT Outcode, YearBin
,AVG(price_paid) AS upperQuartile
FROM (
SELECT Outcode, YearBin
,price_paid
,ROW_NUMBER() OVER (PARTITION BY Outcode, YearBin ORDER BY price_paid) rn
,COUNT(*) OVER (PARTITION BY Outcode, YearBin) cnt
FROM pricePaid
) dd
WHERE rn IN (FLOOR(3 * (cnt + 1) / 4), FLOOR(3 * (cnt + 2) / 4))
GROUP BY Outcode, YearBin;

SELECT t1.*, t2.median, t3.lowerQuartile, t4.upperQuartile, t5.skew
FROM minMaxMeanCountStddev t1
INNER JOIN median t2 on t1.Outcode = t2.Outcode and t1.YearBin = t2.YearBin
INNER JOIN lowerQuartile t3 on t1.Outcode = t3.Outcode and t1.YearBin = t3.YearBin
INNER JOIN upperQuartile t4 on t1.Outcode = t4.Outcode and t1.YearBin = t4.YearBin
INNER JOIN skew t5 on t1.Outcode = t5.Outcode and t1.YearBin = t5.YearBin;
~~~~

Now we can load the aggregated data into Python and additionally prepare a GeoJSON file, which is needed so that the Shiny app knows where the boundaries of each Outcode area are.

**change this code to read aggregated data**

In [ ]:
from pathlib import Path
import pandas as pd

### Load data
appDir = Path(__file__).parent
print("Importing data...")
dataset = pd.read_csv(appDir / "pricepaidsample.csv", delimiter='\t', header=0, encoding="utf-8")
# remove missing postcodes and price_paid values of zero
dataset = dataset[~(dataset['postcode'].isnull() | dataset['price_paid'] == 0)]

A folder containing a GeoJSON file with the polygon coordinates of each Outcode was downloaded online (https://github.com/mhl/postcodes-mapit). This is combined into a single GeoJSON file:

In [ ]:
import json

### Get polygon coordinates (GeoJSON) of each PostcodeArea
geojsonDir = appDir / 'districts'
# combine all PostcodeArea datasets, one for each PostcodeArea
geojsonDict = {}
print("Importing geojson files...")
for file in geojsonDir.glob('*.geojson'):
    with open(file, 'r') as f:
        geojsonDict[str(file).split('\\')[-1][:-8]] = json.load(f)
# add id string to link to summary data
for key in geojsonDict.keys():
    geojsonDict[key]['features'][0]['id'] = key
# changing format of geojsonDict to meet required format for Choropleth function
geojsonList = []
uniqueOutcodes = dataset['Outcode'].unique().tolist()
for outcode in geojsonDict.keys():
    features = geojsonDict[outcode]['features']
    if outcode in uniqueOutcodes:
        geojsonList.append(features[0])
geojsonDict = {}
geojsonDict['type'] = 'FeatureCollection'
geojsonDict['features'] = geojsonList

Summary statistics (mean, median, min, max) of the price paid were calculated for each Outcode and YearBin:

In [ ]:
import itertools

### aggregate data import by Outcode and YearBin
summary = dataset.groupby(["Outcode", "YearBin"])['price_paid'].agg(['min', 'max', 'mean', 'median']).reset_index()
# make sure there is a row for every Outcode and YearBin - set aggregate values to null if missing
uniqueYearBins = dataset['YearBin'].unique().tolist()
crossJoin = list(itertools.product(uniqueOutcodes, uniqueYearBins))
crossJoin = pd.DataFrame(crossJoin, columns=["Outcode", "YearBin"])
summary = pd.merge(crossJoin, summary, how="left", on=["Outcode", "YearBin"])

As the final step in the data preparation, the GeoJSON file and summary dataset were exported to the folder so that they can be read into the Shiny app:

In [ ]:
# export as geojson dictionary as json file
with open(appDir / 'OutcodeCoordinates.json', 'w') as fp:
    json.dump(geojsonDict, fp)

# export summary dataframe as csv
summary.to_csv(appDir / 'summary.csv', index=False)

##### Building the Shiny app

The app.py program of the Shiny app consists of 3 main parts:

- importing the data
- building the HTML interface
- defining a server function

###### Importing the data

In [ ]:
from pathlib import Path
import os
import json
import pandas as pd
from ipyleaflet import Map, Choropleth
from shiny import App, Inputs, Outputs, Session, ui, reactive
from shinywidgets import output_widget, render_widget
from branca.colormap import linear

### Load data
appDir = Path(os.path.abspath(''))
with open(appDir / 'OutcodeCoordinates.json', 'r') as f:
    outcodeCoordinates = json.load(f)
summary = pd.read_csv(appDir / 'summary.csv')
yearBins = list(summary['YearBin'].unique())

###### Building the HTML interface

In [ ]:
# Nest Python functions to build an HTML interface
app_ui = ui.page_fillable( 
    # Layout the UI with Layout Functions
    # Add Inputs with ui.input_*() functions 
    # Add Outputs with ui.output_*() functions
    ui.layout_sidebar(
        ui.sidebar(
            ui.input_checkbox_group('yearBin', "Time Period", yearBins),
            ui.input_select('statistic', "House Price Summary Statistic", ['mean', 'median', 'min', 'max'], selected='mean', multiple=False),
            ui.input_switch("switch", "Compare Summary Statistic between Time Periods", False)
        ),
        ui.card(
            output_widget("map", fill=True),
            full_screen=True,
            fill=True
        ),
        full_screen=True
    ),
    title="House Prices Visualisation"
)

This results in a HTML template that looks like this:

![HTMLTemplate](../images/HTMLTemplate.PNG)

###### Defining a server function

Within the server function, the createChoroData function is reactive and will be called every time one of the inputs to the app changes. The functions uses the user inputs the filter the dataset and return a dictionary containing a key for each Outcode and a value corresponding to the summary statistic.

The createChoroData function also contains logic for if more than one time period is selected - in this case the function will calculate the difference between the summary statistics of the earliest and latest time period selected.

The map function creates an ipyleaflet Map object and adds a Choropleth layer to it.

In [ ]:
# Define server
def server(input: Inputs, output: Outputs, session: Session):
    # function to filter the summary dataset and return a lookup dictionary with a key for each Outcode
    @reactive.calc
    def createChoroData():

        # select all time periods if none are selected
        if input.yearBin() == tuple():
            filter = yearBins
        else:
            filter = list(input.yearBin())

        # logic for comparing summary stastics between time periods
        if input.switch():
            minYearBin = filter[0]
            maxYearBin = filter[-1]
            dfMin = summary[summary['YearBin'] == minYearBin][['Outcode', input.statistic()]]
            dfMax = summary[summary['YearBin'] == maxYearBin][['Outcode', input.statistic()]]
            df = pd.merge(dfMin, dfMax, how="inner", on="Outcode")
            df['diff'] = df[input.statistic() + '_y'] - df[input.statistic() + '_x']
            df = df.set_index('Outcode')
            return df['diff'].to_dict()
        # if not comparing time periods then just show summary statistic
        else:
            df = summary[summary['YearBin'].isin(filter)].set_index('Outcode')
            return df[input.statistic()].to_dict()

    ### For each output, define a function that generates the output
    @render_widget  
    def map():
        # create a Map object and add a Choropleth layer to it
        m = Map(center=(54.00366, -2.547855), zoom=5.5, zoom_snap=0.2)

        layer = Choropleth(
                    geo_data=outcodeCoordinates,
                    choro_data=createChoroData(),
                    key_on='id',
                    colormap=linear.viridis,
                    border_color='black',
                    style={'fillOpacity': 0.8, 'dashArray': '5, 5'}
                )
        
        m.add(layer)

        return m

# Call App() to combine app_ui and server() into an interactive app
app = App(app_ui, server)

##### Transforming the data (again)

The Shiny app is now working, however it is not very insightful because (except for a couple of areas in London) every area appears to be the same colour:

<img src="../images/MapNoTransformation.PNG" alt="MapNoTransformation" width="20px" class="bg-primary">

The density plot below, of all of the house prices in the sample, shows what the problem is. The data is highly right skewed, and because the colour map works by assigning a different colour to each quantile, the majority of data has the same colour.

**need to change this to plot the distribution of the means instead of price_paid**

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

### Load data
appDir = Path(__file__).parent
dataset = pd.read_csv(appDir / "pricepaidsample.csv", delimiter='\t', header=0, encoding="utf-8")
# remove missing postcodes and price_paid values of zero
dataset = dataset[~(dataset['postcode'].isnull() | dataset['price_paid'] == 0)]

dataset['price_paid_mil'] = dataset['price_paid'] / 1000000

fig, ax = plt.subplots(2, 1, figsize=(10, 12))
sm.qqplot(dataset['price_paid_mil'], ax=ax[1])
sns.kdeplot(data=dataset, x='price_paid_mil', ax=ax[0])
ax[0].set_xlabel('Price Paid (£m)')
ax[0].set_xlim(left=0.0)
ax[1].set_ylim(bottom=0.0)
ax[0].set_title('Density Plot of Price Paid Sample (£m)')
ax[1].set_title('QQ Plot of Price Paid Sample (£m)')

plt.tight_layout()
plt.show()

![DensityPlot](../images/DensityPlot.PNG)

To resolve this, the QuantileTransformer function from scikit-learn was used to mathematically transform the data to follow a more normal distribution. This is a non-parametric method that works by mapping each quantile of the input distribution onto those of the output distribution (in this case the normal distribution) and applying the required transformation to each quantile so that they match as closely as possible. This is often used to transform features for machine learning problems because certain machine learning algorithms cannot effectively deal with non-normal features.

In [ ]:
from sklearn.preprocessing import QuantileTransformer
import numpy as np

qt = QuantileTransformer(output_distribution='normal', random_state=0)
values = qt.fit_transform(np.array(list(dataset['price_paid_mil'])).reshape(-1, 1)).flatten()

fig, ax = plt.subplots(2, 1, figsize=(10,12))
sm.qqplot(data=np.array(values), ax=ax[1])
sns.kdeplot(data=values, ax=ax[0])
ax[0].set_xlabel('Price Paid (£m)')
ax[0].set_title('Density Plot of Price Paid Sample with Quantile Transformation (£m)')
ax[1].set_title('QQ Plot of Price Paid Sample with Quantile Transformation (£m)')

plt.tight_layout()
plt.show()

![DensityPlotQuantileTransformed](../images/DensityPlotQuantileTransformed.PNG)

This results in a distribution that is much more normal, so should make more sense when the colour map is applied to it. However, we can see from the QQ plot that the distribution deviates from normality at each extreme.

The following function was added to the Shiny app and applied to the lookup dictionary returned by the createChoroData function.

In [ ]:
def quantileTransformer(dictionary):
    qt = QuantileTransformer(output_distribution='normal', random_state=0)
    values = qt.fit_transform(np.array(list(dictionary.values())).reshape(-1, 1)).flatten()
    return dict(zip(dictionary.keys(), values))

This change to the app produces the map below:

![MapQuantileTransformed](../images/MapQuantileTransformed.PNG)

This is definitely an improvement, as we can now see some colour variation across the map, however the range of colours is still quite narrow. This is due to the long tails shown in the density plot.

Another approach to this problem was to bin the data into deciles, adding the following step to the createChoroData function before returning a dictionary with the decile as the value in each key:value pair:

In [ ]:
df['decile'] = pd.qcut(df[summary_statistic], 10, labels=False) # summary_statistic is either input.statistic() or 'diff'

Using the deciles of the summary statistic gives a much better colour variation across the map, clearly highlighting the regional differences. This method has the added benefit of being conceptually simple, with the top 10% of values being assigned to the brightest colour and bottom 10% being assigned to the darkest colour.

![MapDeciles](../images/MapDeciles.PNG)

##### GeoJSON compression

The app is now working and displaying a useful colour map, however it is extremely slow to respond to any user input. This is because of the large size of the GeoJSON file, which has to be processed every time a change is made to the inputs. An easy work-around to this problem was to use https://mapshaper.org/ to simplify the GeoJSON file, reducing its size using the Visvalingam / weighted area method with a 1% zoom level. The import was then changed to read from this GeoJSON file instead:

In [ ]:
with open(appDir / 'OutcodeCoordinates_compressed.json', 'r') as f:
    outcodeCoordinates = json.load(f)

##### Analysis

What summary statistics are best to describe the data? If I know which distribution the data fits most closely then that would be helpful.

Do different areas follow different types of distribution?

Has the type of distribution changed over time?

**Min, Max**

Looking at all time periods, the min shows a random pattern with no clear trend by area.

**Mean and Median**

**Which areas have had the greatest change in median over time?**

**Which areas have had the greatest change in max over time?**